In [ ]:
# Load the necessary libraries
library(tictoc);library(glue);library(googleCloudStorageR);library(googleAuthR);library(dplyr);library(tidyverse);
library(bigrquery);library(stringr);library(ggplot2);library(gridExtra);library(readr)

In [ ]:
# tic("Total person data loading time")

# # This query represents dataset "All_participants_dataset_022725" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_46654386_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth,
        person.self_reported_category_concept_id,
        p_self_reported_category_concept.concept_name as self_reported_category
    FROM
        `person` person
    LEFT JOIN
        `concept` p_gender_concept
            ON person.gender_concept_id = p_gender_concept.concept_id
    LEFT JOIN
        `concept` p_race_concept
            ON person.race_concept_id = p_race_concept.concept_id
    LEFT JOIN
        `concept` p_ethnicity_concept
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id
    LEFT JOIN
        `concept` p_sex_at_birth_concept
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id
    LEFT JOIN
        `concept` p_self_reported_category_concept
            ON person.self_reported_category_concept_id = p_self_reported_category_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.

# If you want to read the data, you don't have any line in this chunk again except this path.
# Make sure the date is the same as where the data was intially saved.
#--------------------------------------------
person_46654386_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  "20250303",
  "person_46654386",
  "person_46654386_*.csv")
message(str_glue('The data will be written to {person_46654386_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))
#--------------------------------------------
# # Perform the query and export the dataset to Cloud Storage as CSV files.
# # NOTE: You only need to run `bq_table_save` once. After that, you can
# #       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_46654386_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_46654386_path,
  destination_format = "CSV")

toc()

In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_46654386_path}` to copy these files
#       to the Jupyter disk.
tic("Total persons data loading time") #Start overall time
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character(), self_reported_category = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_46654386_person_df <- read_bq_export_from_workspace_bucket(person_46654386_path)

dim(dataset_46654386_person_df)

head(dataset_46654386_person_df, 5)
toc()

In [ ]:
# rename the data
person_dt <- dataset_46654386_person_df

In [ ]:
# # This query represents dataset "All_participants_dataset_022725" for domain "condition" and was generated for
# # All of Us Controlled Tier Dataset v8
tic("Total conditions data loading time") #Start overall time
dataset_46654386_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name
    FROM
        `condition_occurrence` c_occurrence
    LEFT JOIN
        `concept` c_standard_concept
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id
    LEFT JOIN
        `concept` c_type
            ON c_occurrence.condition_type_concept_id = c_type.concept_id
    LEFT JOIN
        `concept` c_source_concept
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id
    LEFT JOIN
        `concept` c_status
            ON c_occurrence.condition_status_concept_id = c_status.concept_id
")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
#--------------------------------------------
condition_46654386_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  "20250228",
  "condition_46654386",
  "condition_46654386_*.csv")
message(str_glue('The data will be written to {condition_46654386_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))
#--------------------------------------------
# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_46654386_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_46654386_path,
  destination_format = "CSV")
toc()

In [ ]:
# Function for condition data
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_022625_path}` to copy these files
#       to the Jupyter disk.
tic("Total conditions data loading time") #Start overall time
read_bq_export_condition <- function(export_path) {
  col_types <- cols(
    standard_concept_name = col_character(),
    standard_concept_code = col_character(),
    standard_vocabulary = col_character(),
    condition_type_concept_name = col_character(),
    condition_source_value = col_character(),
    source_concept_name = col_character(),
    source_concept_code = col_character(),
    source_vocabulary = col_character(),
    condition_status_source_value = col_character(),
    condition_status_concept_name = col_character()
  )
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}

dataset_46654386_condition_df <- read_bq_export_condition(condition_46654386_path)

dim(dataset_46654386_condition_df)

head(dataset_46654386_condition_df, 5)
toc()

In [ ]:
head(dataset_46654386_condition_df, 20)

In [ ]:
# Rename the data
all_conditions_dt <- dataset_46654386_condition_df

In [ ]:
# Distinct number of conditions in the All of Us data
n_distinct(all_conditions_dt$condition_concept_id)

In [ ]:
# Count occurrences of each condition_concept_id
condition_counts <- all_conditions_dt %>%
  group_by(condition_concept_id, standard_concept_name) %>%
  summarise(count = n(), .groups = "drop")

# View the result
head(condition_counts,20)

In [ ]:
nrow(condition_counts)

In [ ]:
# Filter the dataset for "Pre diabetes" with case insensitivity and handling hyphen variations
pre_diabetes_conditions <- all_conditions_dt %>%
  filter(str_detect(tolower(`standard_concept_name`), regex("pre[- ]?diabetes", ignore_case = TRUE))) %>%
  select(`person_id`, `condition_concept_id`, `standard_concept_name`)

# Display the result
print(pre_diabetes_conditions)

In [ ]:
# Distinct number of persons in the Pre-diabetes cohort
n_distinct(pre_diabetes_conditions$person_id)

In [ ]:
# Filter dataset by diabetes condition
# Copied concepts Ids from the data browser in the Researcher workbench
concept_ids_diabetes <- c(201820,201826,4008576,4193704,442793,443732,4034964,37311673,37016349,443730,376065,
                          4082346,192279,4044391,443731,4311708,376112,37016354,43530689,43531578,37017432,443767,
                          443238,443733,321822,443729,201254,4174977,443412,376683,4029423,4226121,195771,435216,
                          45757363,43530656,378743,4058243,4159742,45757435,45757277,43530690,376979,4221495,4175440,
                          443727,42536605,4024659,37016768,40482801,37016348,4009303,37110593,4033942,43531616,4334884,
                          35626904,380097,377821,380096,40484648,45757124,200687,42538169,43530685,377552,4099651,194700,
                          35626070,45770928,45770830,37312218,4227210,45770881,443735,45769876,4226354,4222876,37018566,
                          4196141,43531007,37017431,4202383,4048028,4226238,4114427,40485020,43531563,36717156,443734,
                          4222415,4063043,4129519,43531010,4130162,4131908,37016767,45757129,318712,4029420,376114,45763583,
                          45757079,45763584,36715571,439770,37016179,45770902,37311254,45757474,37311253,443012,4266637,
                          35626764,4290822,4063042,43531009,43531008,35626039,201530,35626038,45757789,761051,45769832,
                          4304377,4225656,37016180,4226798,4191611,4189418,4099214,4152858,37016355,761049,4200875,4338901,
                          4095288,43531564,36674200,4234742,36714116,36674199,4140466,45769830,45773064,35626041,45770880,
                          4223303,4145827,609096,35626042,40484649,45771064,609095,4227657,4225055,609114,45773688,609099,
                          609101,4326434,4062686,609103,761048,37016357,37017430,4206115,4224254,4030664,42535540,45757508,
                          43531653,4228112,42537681,4228443,201531,4128221,35626044,45757449,35626088,4171406,35626068,
                          35626043,35626087,35626067,37016358,3655382,609116,3655383,4263902,609104,609119,4230254,43531651,
                          609108,37311833,609105,36712687,609115,609118,609106,609117,609120,192691,37016356,45769873,
                          37311832,609097,36712686,609107,609100,45757499,4178452,45757674,40480031,35626069,45769905,
                          40480000,4099216,45757266,37018765,4147719,609102,45757507,37312204,609098,443592,609109,4245270,
                          4143857,40482883,35626763,4215719,37312207,43531577,4143529,43531011,4177050,37018728,45757362,
                          4016047,4060085,45757432,609112,602345,4144583,4062685,43531597,4054812,603318)
# Pre-diabetes cohort added. They are the last two concept ids(37018196,44808385)

In [ ]:
# Length of the concept IDs
length(concept_ids_diabetes)

In [ ]:
# Extract diabetes related conditions:
# Extract all conditions containing "diabetes" or "diabetics"
diabetes_conditions <- condition_counts %>%
  filter(grepl("diabetes|diabetic", standard_concept_name, ignore.case = TRUE) &
          !(condition_concept_id %in% c(37018196,44808385,30968,4099334))
)
# View the first few rows
head(diabetes_conditions, 20)

# Count how many diabetes-related conditions were found
nrow(diabetes_conditions)

In [ ]:
# Compare the new concept ids with the existing ones I had
new_diabetes_ids <- setdiff(diabetes_conditions$condition_concept_id, concept_ids_diabetes)
print(new_diabetes_ids)

In [ ]:
# Unique condition concept IDs
concept_ids_diabetes_2 <- unique(diabetes_conditions$condition_concept_id)

In [ ]:
length(unique(concept_ids_diabetes_2))

In [ ]:
# Filter the dataset for rows where condition_concept_id is in the diabetes concept ID list
diabetes_dataset <- all_conditions_dt %>%
  filter(condition_concept_id %in% concept_ids_diabetes_2)

# Display the first few rows of the filtered dataset
head(diabetes_dataset)

In [ ]:
length(unique(diabetes_dataset$condition_concept_id))

In [ ]:
# What is the dimension of the dataset
dim(diabetes_dataset)

# The distinct number of persons in the dataset
n_distinct(diabetes_dataset$person_id)

In [ ]:
# Ensure one row per respondent by keeping only the first occurrence for each person_id
diabetes_dataset_unique <- diabetes_dataset %>%
  arrange(person_id, condition_start_datetime) %>%  # Sort by person_id and earliest date
  group_by(person_id) %>%
  slice(1) %>%  # Select the first occurrence per person (earliest date)
  ungroup()
dim(diabetes_dataset_unique)

In [ ]:
# group data by concept id and concept name
# shows the number of participants that has different conditions
temp1 <-
  diabetes_dataset %>%
  dplyr::group_by(condition_concept_id,standard_concept_name) %>%
  dplyr::summarise(countp = n_distinct(person_id))
temp1

In [ ]:
# Extract diabetes related conditions:
# Extract all conditions containing "diabetic neuropathy" or "polyneuropathy" or neuropathy or neuropathic or foot ulcer
neuropathy_conditions <- condition_counts %>%
   filter(grepl("diabetic neuropathy|polyneuropathy|neuropathy|neuropathic|diabetic foot ulcer", standard_concept_name,
   #filter(grepl("diabetic neuropathy|diabetic foot ulcer|neuropathy due to diabetes", standard_concept_name,
               ignore.case = TRUE))

# View the first few rows
head(neuropathy_conditions, 20)

# Count how many diabetes-related conditions were found
nrow(neuropathy_conditions)

In [ ]:
# Check the un ique neuropathy list
length(unique(neuropathy_conditions$standard_concept_name))

In [ ]:
# Define the standard concept names to exclude
to_be_excluded_conditions <- c(
  "Post-herpetic polyneuropathy", "Idiopathic peripheral autonomic neuropathy", "Mumps polyneuropathy",
  "Toxic polyneuropathy", "Ischemic optic neuropathy", "Critical illness polyneuropathy",
  "Polyneuropathy in collagen vascular disease", "Idiopathic peripheral neuropathy",
  "Polyneuropathy associated with another disorder", "Hereditary peripheral neuropathy",
  "Toxic optic neuropathy", "Inflammatory and toxic neuropathy", "Paraneoplastic neuropathy",
  "Alcoholic polyneuropathy", "Idiopathic progressive polyneuropathy", "Hereditary sensory neuropathy",
  "Nutritional optic neuropathy", "Chronic inflammatory demyelinating polyradiculoneuropathy",
  "Polyneuropathy due to drug", "Femoral neuropathy", "Superficial peroneal neuropathy",
  "Anterior ischemic optic neuropathy of right eye", "Entrapment neuropathy of right common peroneal nerve",
  "Right common peroneal neuropathy at the fibular head", "Right deep peroneal neuropathy",
  "Left common peroneal neuropathy at the fibular head", "Entrapment neuropathy of left sural nerve",
  "Entrapment neuropathy of right plantar nerve", "Ulnar neuropathy of right arm",
  "Ulnar neuropathy of left arm", "Retrobulbar neuropathy", "Deep peroneal neuropathy",
  "Sensory polyneuropathy", "Familial non-neuropathic amyloidosis", "Ischemic peripheral neuropathy",
  "Radial neuropathy", "Idiopathic trigeminal neuropathy",
  "Neuropathy associated with dysproteinemias", "Neuropathy due to infection",
"Idiopathic chronic neuropathy",
  "Demyelinating sensorimotor neuropathy", "Neuropathy in benign monoclonal gammopathy",
  "Motor neuropathy with multiple conduction block", "Neuropathy caused by chemical substance",
  "Intercostal neuropathy", "Axonal sensorimotor neuropathy", "Arteritic ischemic optic neuropathy",
  "Non-arteritic ischemic optic neuropathy", "Neuropathy associated with endocrine disorder",
  "Neuropathy due to human immunodeficiency virus", "Neuropathy caused by organic substance",
  "Compression neuropathy of upper limb", "Compression neuropathy of lower limb",
  "Inflammatory neuropathy", "Peripheral demyelinating neuropathy", "Neuropathic spondylopathy",
  "Polyneuropathy in herpes zoster", "Neuropathy in association with hereditary ataxia",
  "Polyneuropathy in rheumatoid arthritis", "Polyneuropathy due to amyloidosis",
  "Sarcoid neuropathy", "Serum neuropathy", "Polyneuropathy in disseminated lupus erythematosus",
  "Polyradiculoneuropathy", "Mononeuropathy", "Hereditary motor and sensory neuropathy",
  "Peripheral axonal neuropathy", "Sural neuropathy", "Vasculitic neuropathy",
  "Drug induced optic neuropathy", "Tibial neuropathy", "Common peroneal neuropathy",
  "Hereditary sensory-motor neuropathy, type I", "Amyloid polyneuropathy type I",
  "Familial amyloid polyneuropathy", "Vitamin deficiency related neuropathy",
  "Obturator neuropathy", "Lateral plantar neuropathy", "Mixed sensory-motor polyneuropathy",
  "Sciatic neuropathy", "Anterior ischemic optic neuropathy", "Ulnar neuropathy",
  "Upper brachial plexus neuropathy",
  "Axonal neuropathy", "Median neuropathy", "Neuropathic ulcer", "Neuropathy",
  "Acute herpes zoster neuropathy", "Motor polyneuropathy", "Entrapment neuropathy of upper limb",
  "Sensory neuropathy", "Peripheral motor neuropathy",
  "Peripheral neuropathy due to and following antineoplastic therapy",
  "Distal hereditary motor neuropathy type 1", "Immune-mediated neuropathy",
  "Ischemic optic neuropathy of right eye", "Ischemic optic neuropathy of left eye",
  "Bilateral ischemic optic neuropathy of eyes", "Bilateral peripheral neuropathy of lower limbs",
  "Cranial neuropathy due to Herpes zoster", "Idiopathic small fiber peripheral neuropathy",
  "Peripheral neuropathy with sensorineural hearing impairment syndrome",
  "Length-dependent peripheral neuropathy", "Autonomic neuropathy due to disorder of immune function",
  "Peripheral sensory neuropathy", "Acute motor axonal neuropathy", "Neuropathy due to ionizing radiation",
  "Left cervical root neuropathy", "Right common peroneal neuropathy",
  "Left common peroneal neuropathy", "Right leg peripheral neuropathy",
  "Left leg peripheral neuropathy", "Right radial neuropathy", "Left radial neuropathy",
  "Chronic neuropathic pain", "Autonomic neuropathy due to endocrine disease",
  "Chronic central neuropathic pain", "Radiation polyneuropathy", "Right cervical root neuropathy",
  "Common peroneal neuropathy at the fibular head", "Acute inflammatory demyelinating polyneuropathy",
  "Mononeuropathy of upper limb", "Mononeuropathy of lower limb", "Neuropathy of upper limb",
  "Neuropathy of lower limb", "Suprascapular neuropathy", "Entrapment neuropathy of lower limb",
  "Ulnar neuropathy at wrist", "Neuropathy due to vitamin B12 deficiency",
  "Reflex neuropathic bladder", "Polyneuropathy due to systemic sclerosis",
  "Peripheral neuropathy caused by toxin", "Peripheral neuropathy due to inflammation",
  "Peripheral neuropathy due to metabolic disorder",
  "Neuropathic pain due to radiation","Lumbosacral plexus neuropathy","Autonomic neuropathy","Peripheral neuropathic pain",
  "Neuropathic pain","Polyneuropathy", "Secondary peripheral neuropathy", "Small fiber neuropathy")
# Filter out the unwanted conditions from neuropathy_conditions dataset
filtered_neuropathy_conditions <- neuropathy_conditions %>%
  filter(!(standard_concept_name %in% to_be_excluded_conditions))

# View the updated dataset
head(filtered_neuropathy_conditions)

# Count the remaining conditions
nrow(filtered_neuropathy_conditions)

In [ ]:
# unique(filtered_neuropathy_conditions$condition_concept_id)
# # filter dataset by diabetic neuropathy
concept_ids_dn2 <- unique(filtered_neuropathy_conditions$condition_concept_id)
length(concept_ids_dn2)

In [ ]:
# Filter the dataset for rows where condition_concept_id is in the diabetes concept ID list
diabetic_neuro_dataset <- all_conditions_dt %>%
  filter(condition_concept_id %in% concept_ids_dn2)

# Display the first few rows of the filtered dataset
head(diabetic_neuro_dataset)

In [ ]:
# Dinension and distinct number of diabetic neuropathy patients
dim(diabetic_neuro_dataset)
n_distinct(diabetic_neuro_dataset$person_id)

In [ ]:
# Ensure one row per respondent by keeping only the first occurrence for each person_id
diab_neuro_unique <- diabetic_neuro_dataset %>%
  arrange(person_id, condition_start_datetime) %>%  # Sort by person_id and earliest date
  group_by(person_id) %>%
  slice(1) %>%   # Select the first occurrence per person (earliest date)
  ungroup()
dim(diab_neuro_unique)

In [ ]:
# We are interestd in getting only the earliest dates of diabetes or diabetic neuropathy occurrence

# Ensure dates are in proper datetime format
diabetes_dataset <- diabetes_dataset %>%
  mutate(condition_start_datetime = as.POSIXct(condition_start_datetime, format="%Y-%m-%d %H:%M:%S", tz="UTC"))

diabetic_neuro_dataset <- diabetic_neuro_dataset %>%
  mutate(condition_start_datetime = as.POSIXct(condition_start_datetime, format="%Y-%m-%d %H:%M:%S", tz="UTC"))

# Get the earliest diabetes diagnosis per person
earliest_diabetes <- diabetes_dataset %>%
  group_by(person_id) %>%
  summarise(first_diabetes_date = min(condition_start_datetime, na.rm = TRUE))

# Get the earliest neuropathy diagnosis per person
earliest_neuropathy <- diabetic_neuro_dataset %>%
  group_by(person_id) %>%
  summarise(first_neuro_date = min(condition_start_datetime, na.rm = TRUE))

# Merge the two datasets to compare diagnosis dates
filtered_diabetic_neuro <- earliest_neuropathy %>%
  inner_join(earliest_diabetes, by = "person_id") %>%
  filter(first_diabetes_date < first_neuro_date)  # Retain only cases where diabetes precedes neuropathy

# Merge back with full neuropathy dataset to keep all details of selected cases
final_diabetic_neuro_dataset <- diabetic_neuro_dataset %>%
  semi_join(filtered_diabetic_neuro, by = "person_id")

# Display results
dim(final_diabetic_neuro_dataset)  # Check dimensions
n_distinct(final_diabetic_neuro_dataset$person_id)  # Check unique individuals
head(final_diabetic_neuro_dataset)  # View a sample

In [ ]:
# Ensure dates are in proper datetime format
diabetes_dataset <- diabetes_dataset %>%
  mutate(condition_start_datetime = as.POSIXct(condition_start_datetime, format="%Y-%m-%d %H:%M:%S", tz="UTC"))

diabetic_neuro_dataset <- diabetic_neuro_dataset %>%
  mutate(condition_start_datetime = as.POSIXct(condition_start_datetime, format="%Y-%m-%d %H:%M:%S", tz="UTC"))

# Get the earliest diabetes diagnosis per person
earliest_diabetes <- diabetes_dataset %>%
  group_by(person_id) %>%
  summarise(first_diabetes_date = min(condition_start_datetime, na.rm = TRUE))

# Get the earliest neuropathy diagnosis per person
earliest_neuropathy <- diabetic_neuro_dataset %>%
  group_by(person_id) %>%
  summarise(first_neuro_date = min(condition_start_datetime, na.rm = TRUE))

# Merge the two datasets to compare diagnosis dates
merged_dates <- earliest_neuropathy %>%
  left_join(earliest_diabetes, by = "person_id")

# **Included data:** Individuals where diabetes diagnosis precedes neuropathy
included_patients <- merged_dates %>%
  filter(first_diabetes_date < first_neuro_date) %>%
  select(person_id)

# **Excluded data:** Individuals where diabetes does not precede neuropathy or no diabetes record exists
excluded_patients <- merged_dates %>%
  filter(is.na(first_diabetes_date) | first_diabetes_date >= first_neuro_date) %>%
  select(person_id)

# Merge back to get full dataset details
included_diabetic_neuro_dataset <- diabetic_neuro_dataset %>%
  semi_join(included_patients, by = "person_id")

excluded_diabetic_neuro_dataset <- diabetic_neuro_dataset %>%
  semi_join(excluded_patients, by = "person_id")

# Check dataset dimensions
dim(included_diabetic_neuro_dataset)  # Retained dataset
n_distinct(included_diabetic_neuro_dataset$person_id)  # Unique individuals in included dataset

dim(excluded_diabetic_neuro_dataset)  # Excluded dataset
n_distinct(excluded_diabetic_neuro_dataset$person_id)  # Unique individuals in excluded dataset

# **Save both datasets as CSV files**
write.csv(included_diabetic_neuro_dataset, "included_diabetic_neuro.csv", row.names = FALSE)
write.csv(excluded_diabetic_neuro_dataset, "excluded_diabetic_neuro.csv", row.names = FALSE)

# Display sample of included and excluded datasets
head(included_diabetic_neuro_dataset)
head(excluded_diabetic_neuro_dataset)

In [ ]:
# Check to compare diabetes dates
merged_dates

In [ ]:
# filter data by vitamin D deficiency concept IDs (codes checked in the data browser)
concept_id_vitD_def <- c(436070, 4300954, 4300955)

In [ ]:
# Filter the dataset for rows where condition_concept_id is in the vitamin D deficiency concept ID list
vit_D_def_all_cohorts <- all_conditions_dt %>%
  filter(condition_concept_id %in% concept_id_vitD_def)  %>%
  select(person_id) %>%
  distinct()

# Display the first few rows of the filtered dataset
head(vit_D_def_all_cohorts)

In [ ]:
# check the dimension and distinct number of people with vitamin D deficiency
dim(vit_D_def_all_cohorts)
n_distinct(vit_D_def_all_cohorts$person_id)

In [ ]:
# # This code adjusts the one above by counting the number of unique participant who have data for each measurement concept id
# # Filters only those measurement concept id values where at least 35% of all participants have data
# # The total number of participants is determined from the person table

# # Update: This counts a participant as long as they have a row for that measurement_concept_id,
# # even if the value_as_number is missing (i.e., NA). So for Blood Pressure Panel, Computed blood pressure sys & diastolic,
# # Even though the value is missing, the row still exists, so that person is counted toward the 35% threshold.
# # That's how those variables "scaled through."

# # In other words: we're counting presence of the record, not presence of a meaningful value.

# # Start timing
tic("Total measurement data loading time")

dataset_46654386_measurement_sql <- paste("
WITH MeasurementCounts AS (
    SELECT
        measurement_concept_id,
        COUNT(DISTINCT person_id) AS participant_count
    FROM
        `measurement`
    WHERE
        value_as_number IS NOT NULL
    GROUP BY
        measurement_concept_id
    HAVING
        participant_count >= 0.35 * (SELECT COUNT(DISTINCT person_id) FROM `person`) -- 35% threshold
    OR measurement_concept_id IN (3022192, 3004410, 3028288, 3028437, 3007070, 3027114)
)
SELECT
    measurement.person_id,
    measurement.measurement_concept_id,
    m_standard_concept.concept_name AS standard_concept_name,
    m_standard_concept.concept_code AS standard_concept_code,
    m_standard_concept.vocabulary_id AS standard_vocabulary,
    measurement.measurement_datetime,
    measurement.measurement_type_concept_id,
    m_type.concept_name AS measurement_type_concept_name,
    measurement.operator_concept_id,
    m_operator.concept_name AS operator_concept_name,
    measurement.value_as_number,
    measurement.value_as_concept_id,
    m_value.concept_name AS value_as_concept_name,
    measurement.unit_concept_id,
    m_unit.concept_name AS unit_concept_name,
    measurement.visit_occurrence_id,
    m_visit.concept_name AS visit_occurrence_concept_name,
    measurement.measurement_source_value,
    measurement.measurement_source_concept_id,
    m_source_concept.concept_name AS source_concept_name,
    m_source_concept.concept_code AS source_concept_code,
    m_source_concept.vocabulary_id AS source_vocabulary,
    measurement.unit_source_value,
    measurement.value_source_value
FROM
    `measurement` measurement
JOIN
    MeasurementCounts mc ON measurement.measurement_concept_id = mc.measurement_concept_id
LEFT JOIN
    `concept` m_standard_concept ON measurement.measurement_concept_id = m_standard_concept.concept_id
LEFT JOIN
    `concept` m_type ON measurement.measurement_type_concept_id = m_type.concept_id
LEFT JOIN
    `concept` m_operator ON measurement.operator_concept_id = m_operator.concept_id
LEFT JOIN
    `concept` m_value ON measurement.value_as_concept_id = m_value.concept_id
LEFT JOIN
    `concept` m_unit ON measurement.unit_concept_id = m_unit.concept_id
LEFT JOIN
    `concept` m_visit ON measurement.visit_occurrence_id = m_visit.concept_id
LEFT JOIN
    `concept` m_source_concept ON measurement.measurement_source_concept_id = m_source_concept.concept_id"
)

# # Formulate a Cloud Storage destination path for the data exported from BigQuery.
# # NOTE: By default data exported multiple times on the same day will overwrite older copies.
# #       But data exported on a different days will write to a new location so that historical
# #       copies can be kept as the dataset definition is changed.

measurement_46654386_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  #strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "20250414",
  "measurement_46654386",
  "measurement_46654386_*.csv")
message(str_glue('The data will be written to {measurement_46654386_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# # Perform the query and export the dataset to Cloud Storage as CSV files.
# # NOTE: You only need to run `bq_table_save` once. After that, you can
# #       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_46654386_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_46654386_path,
  destination_format = "CSV")
toc()

In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_022625_path}` to copy these files
#       to the Jupyter disk.
tic("Total measurement data loading time")
read_bq_export_measurement <- function(export_path) {
  col_types <- cols(
    standard_concept_name = col_character(),
    standard_concept_code = col_character(),
    standard_vocabulary = col_character(),
    measurement_type_concept_name = col_character(),
    operator_concept_name = col_character(),
    value_as_concept_name = col_character(),
    unit_concept_name = col_character(),
    visit_occurrence_concept_name = col_character(),
    source_concept_name = col_character(),
    source_concept_code = col_character(),
    source_vocabulary = col_character()
  )
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}


dataset_46654386_measurement_df <- read_bq_export_measurement(measurement_46654386_path)

dim(dataset_46654386_measurement_df)

head(dataset_46654386_measurement_df, 5)
toc()

In [ ]:
# Check the dimension
dim(dataset_46654386_measurement_df)

In [ ]:
# Rename the measurement data
measurement_dt <- dataset_46654386_measurement_df

In [ ]:
#group data by measurements
check <-
    measurement_dt %>%
    dplyr::group_by(standard_concept_name) %>%
    dplyr::summarise(countp = n_distinct(person_id))
check

In [ ]:
# View first few rows of data
head(measurement_dt, 20)

In [ ]:
# We would like to see which measurements have multiple unit codes
# Identify measurements with inconsistent units (which need harmonization)
# Know how many unit concept codes to handle per variable before reshaping or analysis
unique_unit_concept_ids <- measurement_dt %>%
  dplyr::select(standard_concept_name, unit_concept_name) %>%
  distinct() %>%
  arrange(standard_concept_name)
unique_unit_concept_ids

In [ ]:
# Count the number of distinct person_id values for each unit_concept_id per measurement
unit_counts_per_measurement <- measurement_dt %>%
  distinct(standard_concept_code, standard_concept_name, person_id, unit_concept_name) %>%
  group_by(standard_concept_code, standard_concept_name, unit_concept_name) %>%
  summarise(count = n(), .groups = "drop") %>%
  arrange(standard_concept_name, desc(count))

# View top 10 rows
head(unit_counts_per_measurement, 10)

In [ ]:
# Instead of counts, lets do proportion
# Count distinct person_id per unit_concept_name
unit_counts_per_measurement <- measurement_dt %>%
  distinct(standard_concept_code, standard_concept_name, person_id, unit_concept_name) %>%
  group_by(standard_concept_code, standard_concept_name, unit_concept_name) %>%
  summarise(count = n(), .groups = "drop")

# Calculate total counts per measurement
total_counts <- unit_counts_per_measurement %>%
  group_by(standard_concept_code, standard_concept_name) %>%
  summarise(total = sum(count), .groups = "drop")

# Join back and compute proportions
unit_props_per_measurement <- unit_counts_per_measurement %>%
  left_join(total_counts, by = c("standard_concept_code", "standard_concept_name")) %>%
  #mutate(proportion = count / total) %>%
mutate(proportion = round(count / total * 100, 1)) %>%
  arrange(standard_concept_name, desc(proportion))

# View top 10 rows
head(unit_props_per_measurement, 10)

In [ ]:
# Check the summary counts for each unit concept id
range_summary_all <- measurement_dt %>%
  group_by(standard_concept_code, standard_concept_name, unit_concept_id, unit_concept_name, unit_source_value) %>%
  summarise(
    count = n(),
    min_value = min(value_as_number, na.rm = TRUE),
    max_value = max(value_as_number, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  arrange(desc(count))

# View top 10
head(range_summary_all, 10)

In [ ]:
# Check through to see number of unique variables we have
unique_variables <- measurement_dt %>%
  distinct(standard_concept_name) %>%
  arrange(standard_concept_name)

In [ ]:
# View
unique_variables

In [ ]:
# create a mapping to change symbols to words - unit standardization
unit_lookup <- tibble::tribble(
  ~original,                                ~standardized,
  "kg/sq. m",                               "kilogram per square meter",

  "milligram per deciliter calculated",     "milligram per deciliter",
  "mg/dl",                                  "milligram per deciliter",
  "gram per deciliter calculated",          "gram per deciliter",

  "percentage unit",                        "percent",
  "percent hemoglobin",                     "percent",
  "percent hemoglobin a1c",                 "percent",
  "percent total protein",                  "percent",
  "percentage of total",                    "percent",
  "percentage total hemoglobin",            "percent",

  "micro unit per liter",                   "microunit per liter",
  "mU/L",                                   "milliunit per liter",
  "mIU/mL",                                 "milli-international unit per liter",
  "uIU/mL",                                 "micro-international unit per milliliter",
  "IU/L",                                   "unit per liter",
  "minute",                                 "per minute"
)
unit_lookup

In [ ]:
# Apply the mapping to your data
tic("Time used in running this chunk is: ")
measurement_normalized <- measurement_dt %>%
  mutate(unit_concept_name = tolower(unit_concept_name)) %>%
  left_join(unit_lookup, by = c("unit_concept_name" = "original")) %>%
  mutate(unit_concept_name = coalesce(standardized, unit_concept_name)) %>%
  select(-standardized)
toc()

dim(measurement_normalized)

In [ ]:
tic("Time used in running this chunk is: ")

# Clean up invalid unit names first
measurement_valid_units <- measurement_normalized %>%
  filter(!unit_concept_name %in% c("na", "no matching concept") & !is.na(unit_concept_name))

# Count how many times each unit appears per measurement
unit_counts <- measurement_valid_units %>%
  filter(!is.na(unit_concept_name)) %>%
  group_by(standard_concept_name, unit_concept_name) %>%
  summarise(n = n(), .groups = "drop")

# For each biomarker, find the most common unit count
unit_thresholds <- unit_counts %>%
  group_by(standard_concept_name) %>%
  summarise(
    max_count = max(n),
    .groups = "drop"
  )
toc()

In [ ]:
tic("Time used in running this chunk is: ")

# Join and calculate 10% threshold, keep only acceptable units
unit_counts_filtered <- unit_counts %>%
  inner_join(unit_thresholds, by = "standard_concept_name") %>%
  mutate(threshold = 0.1 * max_count) %>%
  filter(n >= threshold) %>%
  select(standard_concept_name, unit_concept_name)

# Filter original dataset to keep only valid units
measurement_filtered <- measurement_valid_units %>%
  inner_join(unit_counts_filtered, by = c("standard_concept_name", "unit_concept_name"))
toc()

In [ ]:
# check the dimension and distinct number of people in the measurements data
dim(measurement_filtered);n_distinct(measurement_filtered$person_id)

In [ ]:
# View the data structure
glimpse(measurement_filtered)

In [ ]:
# The next chunk to run is below.
# This part was to verify the distribution of the different units coming from BMI, for example.

In [ ]:
# # I want to see if the BMI variable that is in 2 different units have people who have values for both units
# # Filter for BMI records
# bmi_data <- measurement_filtered %>%
#   filter(standard_concept_name == "Body mass index (BMI) [Ratio]")
# # Count unique units per person
# bmi_unit_check <- bmi_data %>%
#   group_by(person_id) %>%
#   summarize(num_units = n_distinct(unit_concept_name),
#             units = paste(unique(unit_concept_name), collapse = ", ")) %>%
#   filter(num_units > 1)
# #view conflicting records
# View(bmi_unit_check)

In [ ]:
# #Get Full Records for These People
# conflict_ids <- bmi_unit_check$person_id

# bmi_conflicting_rows <- bmi_data %>%
#   filter(person_id %in% conflict_ids) %>%
#   arrange(person_id, measurement_datetime)

# View(bmi_conflicting_rows)

In [ ]:
# colnames(bmi_conflicting_rows)

In [ ]:
# # Now, I want to check for people who have data for only ratio, and not kilogram per square metter
# # Group and summarize by person
# bmi_unit_summary <- bmi_data %>%
#   group_by(person_id) %>%
#   summarize(units = unique(unit_concept_name)) %>%
#   unnest(units) %>%
#   distinct()

# # Find those who only have "ratio"
# # Count units per person
# bmi_unit_count <- bmi_data %>%
#   group_by(person_id) %>%
#   summarize(all_units = list(unique(unit_concept_name))) %>%
#   mutate(
#     only_ratio = map_lgl(all_units, ~ all(.x == "ratio")),
#     includes_kgm2 = map_lgl(all_units, ~ "kilogram per square meter" %in% .x)
#   ) %>%
#   filter(only_ratio == TRUE & includes_kgm2 == FALSE)

# # Get full rows for these people
# only_ratio_ids <- bmi_unit_count$person_id

# bmi_ratio_only <- bmi_data %>%
#   filter(person_id %in% only_ratio_ids)

# View(bmi_ratio_only)

In [ ]:
# # Let’s confirm that "ratio" values look like real BMI:
# summary(bmi_data %>% filter(unit_concept_name == "ratio") %>% pull(value_as_number))

In [ ]:
# Some people have values for both kilogram per square meter and ratio

In [ ]:
# # Create a Summary Table of Units per Measurement
# unit_summary2 <- measurement_filtered %>%
#   group_by(standard_concept_name, unit_concept_name) %>%
#   summarize(
#     count = n(),
#     non_missing = sum(!is.na(value_as_number)),
#     missing_count = sum(is.na(value_as_number)),
#     mean = mean(value_as_number, na.rm = TRUE),
#     sd = sd(value_as_number, na.rm = TRUE),
#     min_value = min(value_as_number, na.rm = TRUE),
#     max_value = max(value_as_number, na.rm = TRUE),
#     median_value = median(value_as_number, na.rm = TRUE),
#     #over_100k = sum(value_as_number > 100000, na.rm = TRUE),
#     .groups = "drop"
#   ) %>%
#   arrange(standard_concept_name, desc(count))

# # View or write to file
# View(unit_summary2)

In [ ]:
# # Filter unit_summary for One Biomarker
# # Choose a single biomarker to inspect
# biomarker_name <- "Cholesterol in LDL [Mass/volume] in Serum or Plasma"

# # Filter unit summary for this biomarker
# albumin_units <- unit_summary2 %>%
#   filter(standard_concept_name == biomarker_name) %>%
#   arrange(desc(count))

# # View the summary of units used
# View(albumin_units)

In [ ]:
# # Filter unit_summary for One Biomarker
# # Choose a single biomarker to inspect
# biomarker_name <- "Body mass index (BMI) [Ratio]"

# # Filter unit summary for this biomarker
# var_units <- unit_summary2 %>%
#   filter(standard_concept_name == biomarker_name) %>%
#   arrange(desc(count))

# # View the summary of units used
# View(var_units)

In [ ]:
# # Plotting BMI Distributions
# # # Filter extreme values to make plot more interpretable
# # bmi_clean <- bmi_data %>%
# #   filter(unit_concept_name %in% c("kilogram per square meter", "ratio")) %>%
# #   filter(value_as_number > 5, value_as_number < 100)

# # No filtering — include all values for selected units
# bmi_all <- bmi_data %>%
#   filter(unit_concept_name %in% c("kilogram per square meter", "ratio"))

# # Plot distribution
# ggplot(bmi_all, aes(x = value_as_number, fill = unit_concept_name)) +
#   geom_density(alpha = 0.5) +
#   scale_x_log10() +
#   labs(
#     title = "BMI Value Distribution by Unit Type (Log Scale)",
#     x = "BMI Value (log 10 scale)",
#     y = "Density",
#     fill = "Unit Type"
#   ) +
#   theme_minimal()

# #compare them side by side
# ggplot(bmi_all, aes(x = value_as_number)) +
#   geom_density(fill = "steelblue", alpha = 0.6) +
#   facet_wrap(~ unit_concept_name, scales = "free") +
#   labs(title = "BMI Distribution per Unit", x = "BMI", y = "Density") +
#   theme_minimal()

In [ ]:
# # If we exclude outliers using different methods, what are the data ranges?
# # Compute 1st–99th percentile range
# bmi_percentile_summary <- bmi_data %>%
#   filter(unit_concept_name %in% c("kilogram per square meter", "ratio")) %>%
#   group_by(unit_concept_name) %>%
#   summarize(
#     p1 = quantile(value_as_number, 0.01, na.rm = TRUE),
#     p99 = quantile(value_as_number, 0.99, na.rm = TRUE),
#     .groups = "drop"
#   )
# print(bmi_percentile_summary)

In [ ]:
# # - value_as_number: BMI values
# # - unit_concept_name: Unit type ("kilogram per square meter", "ratio", etc.)

# # Define percentiles to compute
# percentiles <- c(0.1, 0.5, 99.5, 99.9)

# # Compute percentiles for each unit
# bmi_percentiles <- bmi_data %>%
#   filter(unit_concept_name %in% c("kilogram per square meter", "ratio")) %>%
#   group_by(unit_concept_name) %>%
#   summarize(across(
#     .cols = value_as_number,
#     .fns = list(
#       `0.1th` = ~ quantile(.x, probs = 0.001, na.rm = TRUE),
#       `0.5th` = ~ quantile(.x, probs = 0.005, na.rm = TRUE),
#       `99.5th` = ~ quantile(.x, probs = 0.995, na.rm = TRUE),
#       `99.9th` = ~ quantile(.x, probs = 0.999, na.rm = TRUE)
#     ),
#     .names = "{.fn}"
#   ))

# print(bmi_percentiles)

In [ ]:
# # Compute IQR-based range
# bmi_iqr_summary <- bmi_data %>%
#   filter(unit_concept_name %in% c("kilogram per square meter", "ratio")) %>%
#   group_by(unit_concept_name) %>%
#   summarize(
#     Q1 = quantile(value_as_number, 0.25, na.rm = TRUE),
#     Q3 = quantile(value_as_number, 0.75, na.rm = TRUE),
#     IQR = Q3 - Q1,
#     lower_bound = Q1 - 1.5 * IQR,
#     upper_bound = Q3 + 1.5 * IQR,
#     .groups = "drop"
#   )
# print(bmi_iqr_summary)

In [ ]:
# Now, we are no longer recoding the "ratio" unit
# Exclude BMI values with 'ratio' as the unit, as it is obvious they are from different population
measurement_filtered_again <- measurement_filtered %>%
  filter(!(standard_concept_name == "Body mass index (BMI) [Ratio]" & unit_concept_name == "ratio"))

In [ ]:
# check the dimensionof the remaining measurement data
dim(measurement_filtered_again)

In [ ]:
# Since we have decided not to concern ourself with the ratio unit for BMI
# Let us rename the measurement_filtered_again as measurement_final, so that we can continue our operation on the data
measurement_final <- measurement_filtered_again

In [ ]:
# Now to count the distinct number of rows lost in the unit harmonization
# We have to count per person per variable
# Now compute data loss per variable
pre_counts <- measurement_valid_units %>%
  group_by(standard_concept_name) %>%
  summarise(n_before = n(), .groups = "drop")

post_counts <- measurement_filtered_again %>%
  group_by(standard_concept_name) %>%
  summarise(n_after = n(), .groups = "drop")

# Count distinct person_id before harmonization
pre_persons <- measurement_valid_units %>%
  distinct(standard_concept_name, person_id) %>%
  group_by(standard_concept_name) %>%
  summarise(distinct_before = n(), .groups = "drop")

# Count distinct person_id after harmonization
post_persons <- measurement_filtered_again %>%
  distinct(standard_concept_name, person_id) %>%
  group_by(standard_concept_name) %>%
  summarise(distinct_after = n(), .groups = "drop")

# Join all together
unit_loss_summary <- pre_counts %>%
  left_join(post_counts, by = "standard_concept_name") %>%
  left_join(pre_persons,  by = "standard_concept_name") %>%
  left_join(post_persons, by = "standard_concept_name") %>%
  mutate(
    n_after = replace_na(n_after, 0),
    distinct_after = replace_na(distinct_after, 0),
    n_lost = n_before - n_after,
    pct_lost = round((n_lost / n_before) * 100, 2),
    distinct_lost = distinct_before - distinct_after
  ) %>%
  arrange(desc(n_lost))

In [ ]:
# View data
unit_loss_summary
#write.csv(unit_loss_summary, "before_vs_after_unit_harmonization.csv")

In [ ]:
# Filter out people with pre-diabetic conditions
diabetes_dataset_clean <- diabetes_dataset %>%
  filter(!person_id %in% pre_diabetes_conditions$person_id#,
         #!person_id %in% statin_data$person_id
        )
n_distinct(diabetes_dataset_clean$person_id)

In [ ]:
# loop through every unique measurement variable in standard_concept_name to check the
# time periods between the measurement dates and the diabetes onset date

# Get list of all unique measurement names
measurement_names <- unique(measurement_final$standard_concept_name)

# Define a function to process each measurement type
process_measurement <- function(measure_name) {
  var_filtered <- measurement_final %>%
    filter(standard_concept_name == measure_name) %>%
    mutate(
      measurement_date = as.Date(measurement_datetime),
      var_name = make.names(measure_name)  # safe column names
    ) %>%
    select(person_id, measurement_date, num_value = value_as_number, var_name)

  vars_with_onset <- var_filtered %>%
    inner_join(diabetes_onset, by = "person_id") %>%
    mutate(
      days_diff = as.numeric(difftime(measurement_date, diabetes_onset_date, units = "days")),
      weeks_diff = days_diff / 7,
      months_diff = days_diff / 30.44  # average days per month
    ) %>%
    group_by(person_id, var_name) %>%
    slice_min(order_by = abs(days_diff), n = 1, with_ties = FALSE) %>%
    ungroup()

  return(vars_with_onset)
}

# Apply the function to each measurement and combine results
tic("Time used in running this chunk is: ")
all_measurements_diff <- map_dfr(measurement_names, process_measurement)
toc()

In [ ]:
# View data - this contains the calculation of differences in the date of diabetes onset and the closest
# measurement dates to it by days, weeks and months
all_measurements_diff

In [ ]:
# Ensure datetime and onset date are Date objects
measurement_with_closest_date <- measurement_filtered_again %>%
  mutate(measurement_date = as.Date(measurement_datetime))

In [ ]:
# check the dimension of the diabetes onset
dim(diabetes_onset)

In [ ]:
# Ensure the diabetes onset date is in date format
diabetes_onset <- diabetes_onset %>%
  mutate(diabetes_onset_date = as.Date(diabetes_onset_date))

# Join and calculate time difference
merged <- measurement_with_closest_date %>%
  inner_join(diabetes_onset, by = "person_id") %>%
  mutate(
    time_diff_days = as.numeric(difftime(measurement_date, diabetes_onset_date, units = "days")),
    abs_time_diff = abs(time_diff_days)
  ) #%>%   # Uncomment this line and the next if you want all to be within one week
  # Filter to measurements within ±7 days of onset
  #filter(abs_time_diff <= 7)


# If we want to have the variables in two parts - sensitive to timing and non-sensitive to timing
# We run the chunk above with the last two lines commented and proceed with the following
# This is becasue we are trying to pick some variables within one week while the others are selected within 6 months

# Define variable groups
sensitive_vars <- c("Glucose", "HbA1c", "Total cholesterol", "Triglyceride",
                    "Cholesterol in HDL", "Cholesterol in LDL",
                    "Aspartate aminotransferase", "Alanine aminotransferase", "Leukocytes")

# Filter separately by timing window

# Sensitive: within ±7 days
sensitive_selection <- merged %>%
  filter(standard_concept_name %in% sensitive_vars, abs_time_diff <= 7) %>%
  group_by(person_id, standard_concept_name) %>%
  slice_min(order_by = abs_time_diff, n = 1, with_ties = FALSE) %>%
  ungroup()

# Less sensitive: within ±180 days
less_sensitive_selection <- merged %>%
  filter(!(standard_concept_name %in% sensitive_vars), abs_time_diff <= 180) %>%
  group_by(person_id, standard_concept_name) %>%
  slice_min(order_by = abs_time_diff, n = 1, with_ties = FALSE) %>%
  ungroup()

# Combine both
final_selection <- bind_rows(sensitive_selection, less_sensitive_selection) %>%
  dplyr::select(person_id, standard_concept_name, value_as_number, measurement_date, diabetes_onset_date)  # Add unit_concept_name if needed

In [ ]:
# View data
final_selection

In [ ]:
# This means they had some measurements, but none within the allowed:

#     ±7 days (for sensitive variables), or

#     ±180 days (for less-sensitive variables).

# Even if a participant has measurements, all their records might fall outside the ±7 days or ±180 days windows.
# Participants in merged but excluded due to timing filters
n_distinct(merged$person_id) - n_distinct(final_selection$person_id)

In [ ]:
# The remaining 6923 - 6842 = 81 participants

# These participants are in diabetes_onset but not in measurement_with_closest_date, so they didn’t even make it into merged.

setdiff(diabetes_onset$person_id, measurement_with_closest_date$person_id) %>% length()  # Should be 81

In [ ]:
# check the dimension and distinct number of people
dim(final_selection)
n_distinct(final_selection$person_id)

In [ ]:
# View data
final_selection

In [ ]:
# To ensure one row per person, you must pre-aggregate or filter the dataset before pivoting.
# Let's keep only the row closest to the diabetes onset date per test per person.

# Keep only the closest measurement per person and concept
group_filtered <- final_selection %>%
  mutate(days_diff = abs(as.numeric(difftime(measurement_date, diabetes_onset_date, units = "days")))) %>%
  group_by(person_id, standard_concept_name) %>%
  slice_min(order_by = days_diff, n = 1, with_ties = FALSE) %>%
  ungroup()

# Create a lookup table for unique diabetes onset date per person
# Add diabetes_onset_date back after pivoting
onset_lookup <- group_filtered %>%
  select(person_id, diabetes_onset_date) %>%
  distinct()

In [ ]:
# View data
group_filtered

In [ ]:
# Let's extract just the ID, biomarker name, and the unit information into a new, smaller table.

unit_table <- group2 %>%
  select(person_id, standard_concept_name, unit_concept_name)
unit_table

In [ ]:
# Reshape the new dataframe that includes the units from long to wide - it contains different units per variable
tic("Time used in running this chunk is: ")
wide_group <- pivot_wider(group_filtered[!is.na(group_filtered$value_as_number),]
                              , id_cols = person_id, names_from = standard_concept_name, values_from = value_as_number) %>%
  left_join(onset_lookup, by = "person_id")
toc()
# Display the first few rows of the wide dataframe
dim(wide_group)
head(wide_group)

In [ ]:
# sort(colSums(!is.na(wide_group)), decreasing = TRUE)
# Count non-NA values per column and convert to data frame
# Properly convert the named vector to a data frame
na_counts_df <- colSums(!is.na(wide_group)) %>%
  sort(decreasing = TRUE) %>%
  as.data.frame() %>%
  rownames_to_column(var = "variable") %>%
  rename(non_missing_count = ".")

# View the result
na_counts_df

In [ ]:
#check if one row is the same as one participant
# we need one unique person_id on one row
count_rows_and_pid <- function(df){
  n_pid = n_distinct(df$person_id)
  nrow = nrow(df)

  print(paste0("N participants: ", n_pid))

  print(paste0("N row: ", nrow))
}
count_rows_and_pid(wide_group)

In [ ]:
# check the dimension of the data in each domain
# person data
dim(person_dt)
#conditions data
dim(diabetes_dataset_unique)
dim(final_diabetic_neuro_dataset)
# measurement data
dim(wide_group)
# conditions data
dim(vit_D_def_all_cohorts)

In [ ]:
# Total number of people in the dataset
n_distinct(final_diabetic_neuro_dataset$person_id)

In [ ]:
# Total number of Prediabetic people in the data should be excluded from our final dataset
prediabetic_persons <- all_conditions_dt %>%
  filter(condition_concept_id %in% c(37018196, 44808385)) %>%
  select(person_id) %>%
  distinct()
dim(prediabetic_persons)

In [ ]:
# Remove prediabetic individuals from persons and measurements data
# prediabetics had already been removed from the diabetic and the diabetic neuropathy group

persons_filtered <- person_dt %>%
  filter(!person_id %in% prediabetic_persons$person_id)
#measurements_filtered <- wide_group_with_ldl_values %>% #use measurement_data here if you dont want the different units
measurements_filtered <- wide_group %>%
  filter(!person_id %in% prediabetic_persons$person_id)
vit_D_def_all_cohorts_filtered <- vit_D_def_all_cohorts %>%
  filter(!person_id %in% prediabetic_persons$person_id)

In [ ]:
# Ascertain that the prediabetic patients are not also included in your diabetic cohort
diabetes_dataset_unique_filtered <- diabetes_dataset_unique %>%
    filter(!person_id %in% prediabetic_persons$person_id)#,
           #!person_id %in% statin_data$person_id)
dim(diabetes_dataset_unique_filtered)

# Do same for diabetic neuropathy
diab_neuro_unique_filtered <- final_diabetic_neuro_dataset %>% #diab_neuro_unique %>%
    filter(!person_id %in% prediabetic_persons$person_id)
dim(diab_neuro_unique_filtered)
n_distinct(diab_neuro_unique_filtered$person_id)

In [ ]:
# check the dimension of the data in each domain again
dim(persons_filtered)
dim(diabetes_dataset_unique_filtered)
dim(diab_neuro_unique_filtered)
dim(wide_group)
dim(vit_D_def_all_cohorts_filtered)

In [ ]:
# check the distinct number of people with diabetic neuropathy
n_distinct(diab_neuro_unique_filtered$person_id)

In [ ]:
# # Merge Person Data, Conditions Data, and Measurement Data

master_dataset <- persons_filtered %>%
  #mutate(diabetes = ifelse(person_id %in% diabetes_dataset_unique_filtered$person_id, "Yes", "No"),
    mutate(diabetes = ifelse(person_id %in% wide_group$person_id, "Yes", "No"),
         diabetic_neuropathy = ifelse(person_id %in% diab_neuro_unique_filtered$person_id, "Yes", "No"),
         vitamin_D_deficiency = ifelse(person_id %in% vit_D_def_all_cohorts_filtered$person_id, "Yes", "No")) %>%
  left_join(wide_group, by = "person_id")

# Join onset dates to the master dataset
master_dataset <- master_dataset %>%
  left_join(diabetes_start_dates, by = "person_id") %>%
  left_join(diab_neuro_start_dates, by = "person_id")


dim(master_dataset)
n_distinct(master_dataset$person_id)
head(master_dataset, 20)

In [ ]:
# Save the data into the google bucket

# time the process
tic("Total saving time")
# write to csv
write.csv(master_dataset, file = "20250719_closest_measurement_to_diabetes_onset_data.csv", row.names = FALSE)

#this is the workspace
(WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET'))
(USER <- Sys.getenv('OWNER_EMAIL'))
# Timestamp to be used
(TIMESTAMP <- strftime(now(), '%Y%m%d/%H%M%S'))
REPORT_FOLDER <- str_glue('{WORKSPACE_BUCKET}/reports/{USER}')
(REPORT_DESTINATION <- str_glue('{REPORT_FOLDER}/{TIMESTAMP}/'))
system(str_glue(
    'echo {USER} about to save copies of work from {TIMESTAMP} | gsutil cp - {REPORT_DESTINATION}comment.txt 2>&1'),
       intern = TRUE)
system(str_glue('gsutil cat {REPORT_DESTINATION}comment.txt'), intern = TRUE)

#list.files()

system(str_glue('gsutil -m cp 20250719_closest_measurement_to_diabetes_onset_data.csv {REPORT_DESTINATION} 2>&1'),
       intern = TRUE)
toc()